# **Задание**


🎯 Используя датасет Census Income Dataset (Adult) https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data, выполните следующие задачи:

1.Постройте базовую модель для предсказания уровня дохода (income) без учета дисбаланса классов.

2.Примените различные методы для решения проблемы дисбаланса классов.

3.Оцените и улучшите метрики модели, добиваясь максимальной точности.

В ДЗ также будет оцениваться точность

## Описание датасета

Этот набор данных может быть использован для прогнозирования того, превышает ли годовой доход 50 000 $ в год или нет, на основе демографических данных переписи населения США 1994 года. Набор данных, который мы читаем, содержит 32 561 строку и 14 признаков.

Признаки:

Age: Continuous (numerical).

Workclass: Categorical (e.g., Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked).

Final Weight (fnlwgt): Continuous (numerical).

Education: Categorical (e.g., Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, etc.).

Education-Num: Continuous (numerical representation of education levels).

Marital Status: Categorical (e.g., Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse).

Occupation: Categorical (e.g., Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, etc.).

Relationship: Categorical (e.g., Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried).

Race: Categorical (e.g., White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black).

Sex: Categorical (Female, Male).

Capital Gain: Continuous (numerical).

Capital Loss: Continuous (numerical).

Hours per Week: Continuous (numerical).

Native Country: Categorical (e.g., United-States, Cambodia, England, Puerto-Rico, Canada, Germany, India, Japan, Greece, South, China, Cuba, etc.).

## Загрузка и предобработка данных

In [640]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings("ignore")


In [641]:
!pip install catboost

In [642]:
pip install imbalanced-learn

In [643]:
# Загрузка данных
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
features = ["age", "workclass", "fnlwgt", "education", "education-num", "marital-status",
                "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss",
                "hours-per-week", "native-country", "income"]
data = pd.read_csv(url, header=None, names=features, skipinitialspace=True)
data

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [644]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education-num   32561 non-null  int64 
 5   marital-status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital-gain    32561 non-null  int64 
 11  capital-loss    32561 non-null  int64 
 12  hours-per-week  32561 non-null  int64 
 13  native-country  32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


Достаточно много категориальных переменных, целевая переменная 'income' также категориальная.

In [645]:
data.describe()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


Можно удалить столбцы 'capital-gain', 'capital-loss'





In [646]:
# data.drop(['capital-gain', 'capital-loss'], axis=1, inplace=True)
# data.sample(5)

In [647]:
data.describe(include='object')

,workclass,education,marital-status,occupation,relationship,race,sex,native-country,income
count,32561,32561,32561,32561,32561,32561,32561,32561,32561
unique,9,16,7,15,6,5,2,42,2
top,Private,HS-grad,Married-civ-spouse,Prof-specialty,Husband,White,Male,United-States,<=50K
freq,22696,10501,14976,4140,13193,27816,21790,29170,24720


In [648]:
# Посмотрим какие значения в столбце 'workclass'
data['workclass'].unique()

array(['State-gov', 'Self-emp-not-inc', 'Private', 'Federal-gov',
       'Local-gov', '?', 'Self-emp-inc', 'Without-pay', 'Never-worked'],
      dtype=object)

In [649]:
# Определим сколько значений '?' в столбце 'workclass'
count_question_marks = (data['workclass'] == '?').sum()
print("Количество '?' в столбце 'workclass':", count_question_marks)

Количество '?' в столбце 'workclass': 1836


In [650]:
# Подсчет количества значений '?' во всем DataFrame
count_question_marks = (data == '?').sum().sum()
print("Количество '?' во всем DataFrame:", count_question_marks)

Количество '?' во всем DataFrame: 4262


In [651]:
# Удаление строк, где есть хотя бы одно значение '?'
data_cleaned = data.replace('?', pd.NA).dropna()
print("Количество строк после удаления:", data_cleaned.shape[0])

Количество строк после удаления: 30162


In [652]:
df = data_cleaned

In [653]:
df.describe(include='object')

,workclass,education,marital-status,occupation,relationship,race,sex,native-country,income
count,30162,30162,30162,30162,30162,30162,30162,30162,30162
unique,7,16,7,14,6,5,2,41,2
top,Private,HS-grad,Married-civ-spouse,Prof-specialty,Husband,White,Male,United-States,<=50K
freq,22286,9840,14065,4038,12463,25933,20380,27504,22654


In [654]:
# Преобразование целевой переменной в бинарный формат
df['income'] = df['income'].apply(lambda x: 1 if x == '>50K' else 0)
df['sex'] = df['sex'].apply(lambda x: 1 if x == 'male' else 0)
df['native-country'] = df['native-country'].apply(lambda x: 1 if x == 'United-States' else 0)
df.drop(['education'], axis=1, inplace=True)

# Преобразование категориальных переменных в дамми-переменные
df = pd.get_dummies(df, drop_first=True)

# Разделение на признаки и целевую переменную
X = df.drop('income', axis=1)
y = df['income']

In [655]:
X

,age,fnlwgt,education-num,sex,capital-gain,capital-loss,hours-per-week,native-country,workclass_Local-gov,workclass_Private,...,occupation_Transport-moving,relationship_Not-in-family,relationship_Other-relative,relationship_Own-child,relationship_Unmarried,relationship_Wife,race_Asian-Pac-Islander,race_Black,race_Other,race_White
0,39,77516,13,0,2174,0,40,1,False,False,...,False,True,False,False,False,False,False,False,False,True
1,50,83311,13,0,0,0,13,1,False,False,...,False,False,False,False,False,False,False,False,False,True
2,38,215646,9,0,0,0,40,1,False,True,...,False,True,False,False,False,False,False,False,False,True
3,53,234721,7,0,0,0,40,1,False,True,...,False,False,False,False,False,False,False,True,False,False
4,28,338409,13,0,0,0,40,0,False,True,...,False,False,False,False,False,True,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,257302,12,0,0,0,38,1,False,True,...,False,False,False,False,False,True,False,False,False,True
32557,40,154374,9,0,0,0,40,1,False,True,...,False,False,False,False,False,False,False,False,False,True
32558,58,151910,9,0,0,0,40,1,False,True,...,False,False,False,False,True,False,False,False,False,True
32559,22,201490,9,0,0,0,20,1,False,True,...,False,False,False,True,False,False,False,False,False,True


In [656]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30162 entries, 0 to 32560
Data columns (total 43 columns):
 #   Column                                Non-Null Count  Dtype
---  ------                                --------------  -----
 0   age                                   30162 non-null  int64
 1   fnlwgt                                30162 non-null  int64
 2   education-num                         30162 non-null  int64
 3   sex                                   30162 non-null  int64
 4   capital-gain                          30162 non-null  int64
 5   capital-loss                          30162 non-null  int64
 6   hours-per-week                        30162 non-null  int64
 7   native-country                        30162 non-null  int64
 8   income                                30162 non-null  int64
 9   workclass_Local-gov                   30162 non-null  bool 
 10  workclass_Private                     30162 non-null  bool 
 11  workclass_Self-emp-inc                30162 no

## Построение базовой модели RandomForestClassifier

In [657]:
from sklearn.ensemble import RandomForestClassifier

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучение модели
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Предсказание
y_pred = model.predict(X_test)

# Оценка модели
print("Accuracy:", round(accuracy_score(y_test, y_pred),4))
print(classification_report(y_test, y_pred))


Accuracy: 0.8551
              precision    recall  f1-score   support

           0       0.89      0.93      0.91      4503
           1       0.75      0.65      0.69      1530

    accuracy                           0.86      6033
   macro avg       0.82      0.79      0.80      6033
weighted avg       0.85      0.86      0.85      6033



Класс 0 (доход <= 50K) имеет высокие значения точности, полноты и F1-меры, что указывает на то, что модель хорошо справляется с его предсказанием.

Класс 1 (доход > 50K) имеет более низкие значения точности, полноты и F1-меры, что указывает на то, что модель хуже справляется с его предсказанием.

Это может быть связано с *дисбалансом классов*, так как количество объектов класса 1 (1530) значительно меньше, чем количество объектов класса 0 (4503).

Необходимо применить методы балансировки классов.

In [658]:
# # Получение важности признаков
importances = model.feature_importances_

# # Сортировка признаков по важности
indices = np.argsort(importances)[::-1]

# # Получение имен признаков
feature_names = X_train.columns
feature_names

Index(['age', 'fnlwgt', 'education-num', 'sex', 'capital-gain', 'capital-loss',
       'hours-per-week', 'native-country', 'workclass_Local-gov',
       'workclass_Private', 'workclass_Self-emp-inc',
       'workclass_Self-emp-not-inc', 'workclass_State-gov',
       'workclass_Without-pay', 'marital-status_Married-AF-spouse',
       'marital-status_Married-civ-spouse',
       'marital-status_Married-spouse-absent', 'marital-status_Never-married',
       'marital-status_Separated', 'marital-status_Widowed',
       'occupation_Armed-Forces', 'occupation_Craft-repair',
       'occupation_Exec-managerial', 'occupation_Farming-fishing',
       'occupation_Handlers-cleaners', 'occupation_Machine-op-inspct',
       'occupation_Other-service', 'occupation_Priv-house-serv',
       'occupation_Prof-specialty', 'occupation_Protective-serv',
       'occupation_Sales', 'occupation_Tech-support',
       'occupation_Transport-moving', 'relationship_Not-in-family',
       'relationship_Other-relative'

## Различные методы для решения проблемы дисбаланса классов.


Дисбаланс классов — это проблема, которая возникает при обучении модели
классификации на несбалансированных данных, когда один класс представлен
значительно меньшим количеством примеров, чем другой класс.

Дисбаланс классов приводит к снижению качества модели, так как происходит смещение в сторону предсказания большего класса и не учитывается меньший класс. В результате модель может неверно классифицировать объекты меньшего класса.

Методы для решения дисбаланса:


*   Использование ансамблевых методов
*   Использование пороговой вероятности
*   Взвешивание классов
*   SMOTE (Synthetic Minority Over-sampling Technique)
*   Balanced bag of histogram gradient boosting
*   Undersampling (Уменьшение большего класса)

### Использование ансамблевых методов

In [659]:
# Использование ансамблевых методов

# Обучение модели Random Forest с учетом весов классов
rf_model = RandomForestClassifier(class_weight='balanced', random_state=42)
rf_model.fit(X_train, y_train)

# Предсказание
y_pred_rf = rf_model.predict(X_test)

# Оценка модели
print("Accuracy with Random Forest:", round(accuracy_score(y_test, y_pred_rf),4))
print(classification_report(y_test, y_pred_rf))

Accuracy with Random Forest: 0.8558
              precision    recall  f1-score   support

           0       0.88      0.93      0.91      4503
           1       0.76      0.63      0.69      1530

    accuracy                           0.86      6033
   macro avg       0.82      0.78      0.80      6033
weighted avg       0.85      0.86      0.85      6033



Результаты почти, как у базовой модели, но немного лучше.

### Использование пороговой вероятности

In [660]:
# Получение вероятностей
y_pred_proba = model.predict_proba(X_test)[:, 1]

# Установка порога
threshold = 0.5 # можно настроить
y_pred_custom_threshold = (y_pred_proba >= threshold).astype(int)

# Оценка модели
print("Accuracy with custom threshold:", round(accuracy_score(y_test, y_pred_custom_threshold),4))
print(classification_report(y_test, y_pred_custom_threshold))

Accuracy with custom threshold: 0.856
              precision    recall  f1-score   support

           0       0.89      0.92      0.91      4503
           1       0.74      0.66      0.70      1530

    accuracy                           0.86      6033
   macro avg       0.82      0.79      0.80      6033
weighted avg       0.85      0.86      0.85      6033



Результаты почти, как у базовой модели, и немного лучше для 1 класса.

### Взвешивание классов

In [661]:
# Взвешивание классов
class_weights = {0: 1, 1: 10}  # Пример весов, можно настроить

# Обучение модели с учетом весов классов
model_cw_rf = RandomForestClassifier(random_state=42, class_weight=class_weights)
model_cw_rf.fit(X_train, y_train)

# Предсказание
y_pred_cw_rf = model_cw_rf.predict(X_test)

# Оценка модели
print("Accuracy:", round(accuracy_score(y_test, y_pred_cw_rf), 4))
print(classification_report(y_test,  y_pred_cw_rf))

Accuracy: 0.8541
              precision    recall  f1-score   support

           0       0.88      0.93      0.90      4503
           1       0.75      0.63      0.69      1530

    accuracy                           0.85      6033
   macro avg       0.82      0.78      0.80      6033
weighted avg       0.85      0.85      0.85      6033



Результаты почти, как у базовой модели.

### Использование SMOTE

In [662]:
from imblearn.over_sampling import SMOTE

# Применение SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Обучение модели на сбалансированных данных
model_resampled = RandomForestClassifier(random_state=42)
model_resampled.fit(X_resampled, y_resampled)

# Предсказание и оценка модели
y_pred_resampled = model_resampled.predict(X_test)
print("Accuracy after SMOTE:", round(accuracy_score(y_test, y_pred_resampled),4))
print(classification_report(y_test, y_pred_resampled))


Accuracy after SMOTE: 0.8473
              precision    recall  f1-score   support

           0       0.89      0.90      0.90      4503
           1       0.71      0.68      0.69      1530

    accuracy                           0.85      6033
   macro avg       0.80      0.79      0.80      6033
weighted avg       0.85      0.85      0.85      6033



Ухудшились немного результаты для класса 1.

### Balanced bag of histogram gradient boosting

In [663]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.ensemble import BalancedBaggingClassifier

X_processed = X

# Приведение имен столбцов к строковому типу
X_processed.columns = X_processed.columns.astype(str)

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42, stratify=y)

# Инициализация SMOTE
smote = SMOTE(random_state=42)

# Применение SMOTE для создания сбалансированных данных
X_resampled_bbcgistgbc, y_resampled_bbcgistgbc = smote.fit_resample(X_train, y_train)

# Создание модели BalancedBaggingClassifier с HistGradientBoostingClassifier
model_bbcgistgbc = BalancedBaggingClassifier(
    estimator=HistGradientBoostingClassifier(random_state=42),
    n_estimators=10,
    random_state=42,
    n_jobs=2
)

# Обучение модели на сбалансированных данных
model_bbcgistgbc.fit(X_resampled_bbcgistgbc, y_resampled_bbcgistgbc)

# Предсказание и оценка модели
y_pred_bbcgistgbc = model_bbcgistgbc.predict(X_test)
print("Accuracy after BBC:", round(accuracy_score(y_test, y_pred_bbcgistgbc), 4))
print(classification_report(y_test, y_pred_bbcgistgbc))


Accuracy after BBC: 0.8556
              precision    recall  f1-score   support

           0       0.90      0.91      0.90      4531
           1       0.72      0.69      0.70      1502

    accuracy                           0.86      6033
   macro avg       0.81      0.80      0.80      6033
weighted avg       0.85      0.86      0.85      6033



Результаты немного улучшились по сравнению  с базовой моделью.

### Undersampling (Уменьшение большего класса)

In [664]:
from imblearn.under_sampling import RandomUnderSampler

# Применение RandomUnderSampler для уменьшения большего класса
undersampler = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = undersampler.fit_resample(X, y)

# Обучение модели на сбалансированных данных
model_resampled = RandomForestClassifier(random_state=42)
model_resampled.fit(X_resampled, y_resampled)

# Предсказание и оценка модели
y_pred_resampled = model_resampled.predict(X_test)
print("Accuracy after RandomUnderSampler:", round(accuracy_score(y_test, y_pred_resampled),4))
print(classification_report(y_test, y_pred_resampled))


Accuracy after RandomUnderSampler: 0.8616
              precision    recall  f1-score   support

           0       1.00      0.82      0.90      4531
           1       0.64      1.00      0.78      1502

    accuracy                           0.86      6033
   macro avg       0.82      0.91      0.84      6033
weighted avg       0.91      0.86      0.87      6033



Улучшилось качество для класса 0, ухудшилось для класса 1 по сравнению с базовым.

## Оцените и улучшите метрики модели, добиваясь максимальной точности.

Для улучшения метрик модели Undersampling (Уменьшение большего класса) будем использовать метод пороговой вероятности.

In [665]:
# Получение вероятностей
y_pred_proba = model_resampled.predict_proba(X_test)[:, 1]

# Установка порога
threshold = 0.75 # можно настроить
y_pred_custom_threshold = (y_pred_proba >= threshold).astype(int)

# Оценка модели
print("Accuracy with custom threshold:", round(accuracy_score(y_test, y_pred_custom_threshold),4))
print(classification_report(y_test, y_pred_custom_threshold))

Accuracy with custom threshold: 0.9219
              precision    recall  f1-score   support

           0       0.97      0.92      0.95      4531
           1       0.80      0.92      0.85      1502

    accuracy                           0.92      6033
   macro avg       0.88      0.92      0.90      6033
weighted avg       0.93      0.92      0.92      6033



 Accuracy: 0.9219 (или 92.19%) — это общая точность модели, что означает, что 92.19% всех предсказаний были правильными. Это высокий уровень точности.

 Результаты показывают, что модель работает хорошо, но есть возможности для дальнейшего улучшения, особенно для меньшего класса 1.